In [18]:
#pip install pymupdf
#!pip install pdfplumber
#pip install tqdm
#!pip install spacy
#!pip install nltk
#python -m spacy download en_core_web_sm
# %pip install spacy
# import spacy
# spacy.cli.download("en_core_web_sm")
# nlp = spacy.load("en_core_web_sm")
# doc = nlp("Apple is looking at buying U.K. startup for $1 billion")
# print([chunk.text for chunk in doc.noun_chunks])
#!pip install sentence_transformers


In [19]:
path_to_pdf = "Animal-Kingdom.pdf"


In [20]:
from random import randint,random
import fitz  # PyMuPDF
from tqdm.auto import tqdm
#read PDF using PyMuPDF
x  = randint(1, 100)
doc = fitz.open(path_to_pdf)  # open a document
page = doc.load_page(x) # number of page  
text = page.get_text("text")  # get plain text

def text_formatter(text: str) -> str:
    text = text.replace("\n", " ")  # replace new line characters with space
    text = " ".join(text.split())  # remove multiple spaces
    text = text.replace("⚪" , "")  # remove special characters
    if text and text[0].isdigit():
        text = text.split(" ", 1)[1] # remove first page number if exists 
    return text

text_formatter(text)

'Non-Chordata Classification of Hemichordata y It has been divided into the following two classes on the basis of body forms:  Class Enteropneusta: Examples: Balanoglossus, Saccoglossus  Class Pterobranchia: Example: Rhabdopleura Distinctive features of Hemichordata y Presence of stomochord y Presence of notochord-like structure known as buccal diverticulum.'

In [21]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.tokenizer
nlp.add_pipe("sentencizer")
pdf_list = []
for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
    text = page.get_text()  # get plain text encoded as UTF-8
    text = text_formatter(text)
    sentences = text.split('. ')
    pdf_list.append({"page_number": page_number, 
                     "char_count": len(text),
                     "Work Counts": len(text.split(" ")), 
                     "page_token_count": len(text) / 4,
                     "text": text,})
    
pdf_list[randint(1,100)]

0it [00:00, ?it/s]

{'page_number': 87,
 'char_count': 582,
 'Work Counts': 67,
 'page_token_count': 145.5,
 'text': 'Chordata Summary: Difference Between Non-Chordata and Chordata Features Invertebrates (Non-Chordates) Vertebrates (Chordates) Symmetry Radial, biradial or lacking Bilateral Metamerism True, pseudometamerism or lacking Internal segmentation Post-anal tail Lacking Usually present projecting beyond anus Level/Grade of organisation Protoplasmic to organ- system Organ-system Germ layers 2(diploblastic), 3(triploblastic) or lacking 3(triploblastic) Coelom Acoelomate, pseudocoelomate or truly coelomate Truly coelomate Gill slits Pharynx perforated by gill slits Gill slits are absent'}

In [22]:
for item in tqdm(pdf_list):
    item["sentences"] = list(nlp(item["text"]).sents)
    #replace "" with "" in each sentence
    item["sentences"] = [str(sentence).replace("", "") for sentence in item["sentences"]]
    item["page_sentence_count_spacy"] = len(item["sentences"])
    
pdf_list[randint(1,100)]

  0%|          | 0/99 [00:00<?, ?it/s]

{'page_number': 39,
 'char_count': 881,
 'Work Counts': 127,
 'page_token_count': 220.25,
 'text': 'Non-Chordata Distinctive Features Over Cnidaria Comb like ciliary plates Bioluminescence PHYLUM PLATYHELMINTHES (FLAT WORMS) y The term Platyhelminthes was given by Gegenbaur. Platy-flat;helminth-worms. y Habitat: They are mostly parasitic and a few are free-living like Dugesia. y Level of Organisation: They have an organ-system level of organisation. The body is dorsoventrally flat and hence known as flatworms. The body is usually unsegmented like in Liver fluke (Fasciola hepatica) except in Tapeworm (Taenia solium). Keywords \x8a Metagenesis \x8a Cnidoblasts \x8a Bioluminescence \x8a Colloblasts y Symmetry: They have bilateral symmetry (first bilateral animal). y Germ Layer: They are first triploblastic organisms. y Body structure: The space between the body wall is filled with parenchyma. The fluid present in Rack your Brain Which proglottides of tapeworm show apolysis and why?',
 'se

In [23]:
def extract_keywords(text: str) -> list[str]:
    """Return a list of keywords from text (noun chunks + named entities)."""
    doc = nlp(text)
    
    # Collect noun chunks
    chunks = [chunk.text for chunk in doc.noun_chunks]
    
    # Collect named entities
    ents = [ent.text for ent in doc.ents]
    
    # Combine & deduplicate while preserving order
    seen = set()
    keywords = []
    for kw in chunks + ents:
        if kw not in seen:
            seen.add(kw)
            keywords.append(kw)
    
    return keywords

In [25]:
import spacy
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 10 
nlp = spacy.load("en_core_web_sm")

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pdf_list):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])
    item["Key_works"] = extract_keywords(item["text"])
    
    
    
pdf_list[randint(1,100)]

  0%|          | 0/99 [00:00<?, ?it/s]

{'page_number': 8,
 'char_count': 1369,
 'Work Counts': 203,
 'page_token_count': 342.25,
 'text': 'Animal Kingdom – Characteristics of Classification  Protostomes: ‘Proto’ means first and ‘stoma’ means mouth. Thus, in protostome organisms, the development of the mouth in the embryo takes place first followed by the anus. Examples: Platyhelminthes, Aschelminthes, Annelida, Mollusca  Deuterostomes: ‘Deutero’ means second, and ‘stoma’ means mouth. Thus, in deuterostome organisms, the development of the anus takes place first and development of mouth in the embryo takes place after it. Examples: Echinodermata, Hemichordata, Chordata Germ Layers The layer of the gastrula which gives rise to the different parts of the body is called the germ layers. On the basis of the number of the layers, there are two categories: y Diploblastic: Two germ layers i.e., ectoderm and endoderm give rise to the different parts of the body. In between the two layers is present a non- cellular layer known as mes

In [34]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pdf_list):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)


  0%|          | 0/99 [00:00<?, ?it/s]

168

In [39]:
import pandas as pd
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(5).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')
    
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

Chunk token count: 23.75 | Text: An anticoagulant hirudin is present in salivary glands of blood sucking leeches (Sanguivorous).
Chunk token count: 19.75 | Text: y Nervous System: Skull is monocondylic i.e. consists of one occipital condyle.
Chunk token count: 21.25 | Text: Cuticle prevents desiccation of the body and provides protection. The glandular cells
Chunk token count: 28.5 | Text: Examples: In normal condition, earthworm secretes urea but it secretes ammonia when sufficient water is available.
Chunk token count: 28.75 | Text: Sting Ray-Its tail consists of barbed spines. Each spine contains poison and can make a severe wound on the victim.


[{'page_number': 4,
  'sentence_chunk': 'Animal Kingdom INTRODUCTION There is a vast diversity in the number and types of animals in nature. They range from unicellular to multicellular. They are found in the deepest of the oceans, in snow-covered mountains, from the poles to the equator. There are differences in the structure and forms of the different animals. But there are some fundamental features in them which show resemblance such as arrangement of cells, body plan, symmetry, segmentation, coelom, germ layer, body temperature, skeleton and notochord, types of digestive, respiratory, circulatory, excretory, and reproductive systems. We need to classify organisms to make our study easier and to study the interrelationship between different groups. BASES OF CLASSIFICATION Habitat y On the basis of habitat, animals are divided into the following types: Aquatic-Organisms that are found in water. They can be: Marine-Echinoderms, many sponges and coelenterates. Fresh water-Prawns, some 

In [43]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2", 
                                      device="cpu") 

for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])



  0%|          | 0/152 [00:00<?, ?it/s]

In [51]:
pages_and_chunks_over_min_token_len[randint(1,100)]

{'page_number': 9,
 'sentence_chunk': 'Animal Kingdom – Characteristics of Classification y Triploblastic: Three germ layers i.e., ectoderm, mesoderm and endoderm give rise to the different parts of the body. Examples: Flatworm to mammals Segmentation Segmentation is the division of a body into portions known as segments. It is of four types: y Metameric segments (True segmentation): The body is divided into segments; internally and externally. Example: Annelida y External Segmentation- Arthropoda y Internal Segmentation- Chordata y Pseudometameric segments (False segmentation): The segments of the body are not formed from the embryo and thus are known as pseudometamerism. Example: Tape worm Previous Year’s Question Which one of the following kinds of animals are triploblastic? (1) Flatworms (2) Sponges (3) Ctenophores (4) Corals',
 'chunk_char_count': 802,
 'chunk_word_count': 117,
 'chunk_token_count': 200.5,
 'embedding': array([ 2.07140800e-02, -5.88311143e-02, -1.13677019e-02, -2.

In [69]:
query = "What is a phylum ?"
query_embedding = embedding_model.encode(query)

#check similarity
import numpy as np
from numpy.linalg import norm
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return np.dot(a, b) / (norm(a) * norm(b))
# Get similarity score for each chunk
for item in pages_and_chunks_over_min_token_len:
    item["similarity"] = cosine_similarity(query_embedding, item["embedding"])
# Sort by similarity score
pages_and_chunks_over_min_token_len = sorted(pages_and_chunks_over_min_token_len,
                                                key=lambda x: x["similarity"], 
                                                reverse=True)
# Show top 5 most similar chunks
pages_and_chunks_over_min_token_len[:5]

[{'page_number': 50,
  'sentence_chunk': 'y Examples: Lumbricus Pheretima -Earthworm Hirudinaria -Bloodsucking leech Classification of Phylum Annelida y Phylum Annelida is classified into the following four classes, on the basis of the presence or absence of parapodia, setae, metamers: Class 1 – Polychaeta – Segmentation is internal or external. Many setae on the lateral parapodia. Examples – Nereis, Aphrodita Class 2 – Oligochaeta – Segmentation is internal and external. Few Setae. Examples – Lumbricus, Nais Class 3 – Hirudinea-Segmentation is external without internal septa. Setae absent. Example – Leech Class 4 – Archiannelida-Segmentation is internal. No setae. Examples – Dinophilus, Protodrilus.',
  'chunk_char_count': 667,
  'chunk_word_count': 96,
  'chunk_token_count': 166.75,
  'embedding': array([ 1.62026566e-02, -2.61489656e-02,  1.71601269e-02, -2.23048893e-03,
         -5.64138056e-04, -1.33825159e-02,  5.12100086e-02, -1.39781479e-02,
         -4.33105528e-02, -6.74154684